In [370]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder
from sklearn.impute import SimpleImputer

In [371]:
df = pd.read_csv('NPPE1_Preprocessing1.csv')
df.head()

,CRIM,ZN,INDUS,POLINDEX,RM,AGE,DIS,HIGHWAYCOUNT,TAX,PTRATIO,IMM,BPL,PRICE,RIVERSIDE
0,1.026769,1.429034,7.851300,1.134216,6.0,42.0,5.251911,5,279.201277,20.689586,398.811960,10.461456,22.991633,NO
1,0.848089,0.255543,6.263434,1.245993,7.0,63.0,4.305546,8,307.444529,17.465398,377.153649,11.619690,24.551055,NO
2,10.925905,0.441022,18.322960,2.824833,8.0,-2.0,2.409495,25,666.492973,20.351601,387.061355,19.366070,15.875346,NO
3,0.559027,1.041175,11.114920,0.794952,6.0,9.0,6.898669,4,305.514181,19.787314,391.778647,6.206820,23.007756,NO
4,0.905063,81.167963,3.673369,1.029030,8.0,20.0,10.246463,1,315.913960,17.360439,395.833166,10.827105,21.503177,NO


In [372]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   CRIM          4000 non-null   float64
 1   ZN            4000 non-null   float64
 2   INDUS         4000 non-null   float64
 3   POLINDEX      4000 non-null   float64
 4   RM            4000 non-null   float64
 5   AGE           4000 non-null   float64
 6   DIS           4000 non-null   float64
 7   HIGHWAYCOUNT  4000 non-null   int64  
 8   TAX           4000 non-null   float64
 9   PTRATIO       4000 non-null   float64
 10  IMM           4000 non-null   float64
 11  BPL           4000 non-null   float64
 12  PRICE         4000 non-null   float64
 13  RIVERSIDE     4000 non-null   object 
dtypes: float64(12), int64(1), object(1)
memory usage: 437.6+ KB


In [373]:
df.describe()

,CRIM,ZN,INDUS,POLINDEX,RM,AGE,DIS,HIGHWAYCOUNT,TAX,PTRATIO,IMM,BPL,PRICE
count,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000,4000.000000
mean,2.174158,12.734715,11.208752,1.331900,7.044500,66.005000,4.820784,8.464000,376.932036,19.051638,370.615373,12.326057,24.355923
std,2.475646,23.312649,6.827509,0.601024,1.240526,29.100923,2.174427,7.460169,150.270818,2.267293,67.687586,6.143060,8.633523
min,0.031093,0.000281,0.633388,0.405085,-1.000000,-2.000000,1.163082,1.000000,187.025099,12.641899,0.532980,1.986295,6.348266
25%,0.622728,0.464314,5.705110,0.864207,6.000000,41.000000,3.117734,4.000000,277.418380,17.634064,378.426323,7.514027,19.259094
50%,1.227945,1.068245,8.906823,1.218840,7.000000,74.000000,4.325631,6.000000,308.255685,19.386282,392.915025,11.188226,22.634566
75%,2.377738,20.323531,18.509019,1.694232,8.000000,93.000000,6.197419,7.000000,405.418015,20.809079,396.962724,16.092990,26.947036
max,12.619405,101.670740,30.367209,4.156570,11.000000,103.000000,14.045318,27.000000,713.028133,24.322091,400.780583,35.997281,53.090256


Q. How many samples are there in the dataset?

In [374]:
print(df.shape[0])

4000


Q. what is the average house price(in lacs)?

In [375]:
print(df["PRICE"].mean())

24.355923220694248


Q. How many houses have 5 or more rooms?

In [376]:
print((df["RM"]>=5).sum())

3953


Q. What is the average price of top 10 most expensive houses?

In [377]:
L = list(df["PRICE"])
L = sorted(L)[-10:]
print((pd.DataFrame(L,columns=['PRICE'])["PRICE"]).mean())

52.36590175716407


Q. What is the total number of missing or unknown values in the number of rooms feature?

In [378]:
print(df["RM"].value_counts())

RM
 7.0     1733
 6.0      961
 8.0      894
 9.0      276
 5.0       47
 10.0      40
-1.0       40
 4.0        7
 11.0       2
Name: count, dtype: int64


Q. What is the total number of missing or unknown values in the age feature?

In [379]:
L = df["AGE"].copy()
# print(type(L))
L.sort_values(inplace=True)
# print(L)
print((L).value_counts()[-2])

50


Q. What is the total number of missing or unknown value in the RIVERSIDE feature?

In [380]:
print(df["RIVERSIDE"].value_counts())

RIVERSIDE
NO         3595
YES         317
UNKNOWN      88
Name: count, dtype: int64


Q. How many houses are on riverside and were built within the last 50 years (i.e. a house 50 years old or younger)? For this question, ignore the rows that have missing values in either riverside feature or age feature.

In [381]:
print(((df["RIVERSIDE"]=="YES") & (df["AGE"]<=50) & (df['AGE']!=-2)).sum())

44


Q. How many houses are near to exactly 6, 7 or 8 highways (all three inclusive)?

In [382]:
print(((df["HIGHWAYCOUNT"]>=6)&(df["HIGHWAYCOUNT"]<=8)).sum())

1211


Q. Create a column 'CATEGORY' and divide the houses in categories as following:

Category 1: house price <10 lacs
Category 2: 10 lacs <= house price <20 lacs
Category 3: 20 lacs <= house price <30 lacs
Category 4: 30 lacs <= house price <40 lacs
Category 5: house price >=40 lacs

Which category has the highest number of records?

In [383]:
df["CATEGORY"] = 5
df["CATEGORY"] -= 1*(df["PRICE"]<40)
df["CATEGORY"] -= 1*(df["PRICE"]<30)
df["CATEGORY"] -= 1*(df["PRICE"]<20)
df["CATEGORY"] -= 1*(df["PRICE"]<10)
print(df["CATEGORY"].value_counts())

CATEGORY
3    2028
2    1158
4     503
5     268
1      43
Name: count, dtype: int64


Q. Apply Pre processing
Divide the data into training and test sets

1. Replace the respective missing or unknown values in features room count, riverside and age with np.nan.
2. Keep 30% of the data as test set.
3. Use random_state as 0
4. PRICE is the target, rest of the columns are the features.
5. Apply train test split.
Hint: look for the documentation of the usual function that divides the data into training and test datasets.

What is the number of samples in the training set?


In [384]:
df["RM"] = df["RM"].replace(-1,np.nan)
df["RIVERSIDE"] = df["RIVERSIDE"].replace("UNKNOWN",np.nan)
df["AGE"] = df["AGE"].replace(-2,np.nan)
Y = df["PRICE"]
L = list(df.columns)
L.remove("PRICE")
X = df[L]
# print(X.head())
# print(Y)
x_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=0)
print(x_train.shape)

(2800, 14)


In [385]:
df.columns

Index(['CRIM', 'ZN', 'INDUS', 'POLINDEX', 'RM', 'AGE', 'DIS', 'HIGHWAYCOUNT',
       'TAX', 'PTRATIO', 'IMM', 'BPL', 'PRICE', 'RIVERSIDE', 'CATEGORY'],
      dtype='object')

In [386]:
x_train.drop("CATEGORY",axis=1,inplace=True)
X_test.drop("CATEGORY",axis=1,inplace=True)

In [387]:
minmax = Pipeline([
    ('2',MinMaxScaler())
])
stand = Pipeline([
    ('4',StandardScaler())
])
imp = Pipeline([
    ('12',SimpleImputer(strategy='median')),
    ('12.1',MinMaxScaler())
])
imp1 = Pipeline([
    ('13',SimpleImputer(strategy='mean')),
    ('13.1',MinMaxScaler())
])
imp2 = Pipeline([
    ('14',SimpleImputer(strategy='most_frequent')),
    ('14.1',OneHotEncoder())
])

In [388]:
ct = ColumnTransformer(transformers=[
    ('1',minmax,['CRIM','ZN','POLINDEX','DIS','HIGHWAYCOUNT','TAX','PTRATIO','IMM','BPL']),
    ('2',stand,['INDUS']),
    ('3',imp,['RM']),
    ('4',imp1,['AGE']),
    ('5',imp2,['RIVERSIDE'])
],remainder='passthrough')

In [389]:
ct

ColumnTransformer(remainder='passthrough',
                  transformers=[('1', Pipeline(steps=[('2', MinMaxScaler())]),
                                 ['CRIM', 'ZN', 'POLINDEX', 'DIS',
                                  'HIGHWAYCOUNT', 'TAX', 'PTRATIO', 'IMM',
                                  'BPL']),
                                ('2', Pipeline(steps=[('4', StandardScaler())]),
                                 ['INDUS']),
                                ('3',
                                 Pipeline(steps=[('12',
                                                  SimpleImputer(strategy='median')),
                                                 ('12.1', MinMaxScaler())]),
                                 ['RM']),
                                ('4',
                                 Pipeline(steps=[('13', SimpleImputer()),
                                                 ('13.1', MinMaxScaler())]),
                                 ['AGE']),
                                ('5',
                                 Pipeline(steps=[('14',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('14.1', OneHotEncoder())]),
                                 ['RIVERSIDE'])])

In [390]:
x_train = ct.fit_transform(x_train)
x_train = pd.DataFrame(x_train, columns=ct.get_feature_names_out())
X_test = ct.transform(X_test)
k1 = (X_test.mean())
X_test = pd.DataFrame(x_train, columns=ct.get_feature_names_out())
k2 = (X_test.mean().mean())

In [391]:
x_train.shape[1]

14

In [392]:
x_train

,1__CRIM,1__ZN,1__POLINDEX,1__DIS,1__HIGHWAYCOUNT,1__TAX,1__PTRATIO,1__IMM,1__BPL,2__INDUS,3__RM,4__AGE,5__RIVERSIDE_NO,5__RIVERSIDE_YES
0,0.189700,0.004561,0.033842,0.124347,0.192308,0.374853,0.829378,0.989745,0.419050,-0.239852,0.428571,0.860000,1.0,0.0
1,0.036362,0.202330,0.162887,0.271862,0.115385,0.069331,0.690556,0.976549,0.140763,-0.529958,0.571429,0.640453,0.0,1.0
2,0.531739,0.010602,0.652160,0.085051,0.884615,0.912012,0.792344,0.937695,0.047807,1.194743,0.142857,0.970000,1.0,0.0
3,0.358052,0.009848,0.301953,0.067032,0.192308,0.410736,0.214812,0.658560,0.436328,1.289528,0.285714,0.960000,1.0,0.0
4,0.110154,0.017579,0.158678,0.325495,0.192308,0.223330,0.658217,0.992762,0.324922,-0.038482,0.714286,0.560000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2795,0.108279,0.006169,0.271625,0.137714,0.153846,0.411110,0.283893,0.860627,0.306643,1.333493,0.285714,0.860000,0.0,1.0
2796,0.038962,0.005680,0.179231,0.490348,0.153846,0.191885,0.709585,0.995029,0.149164,-0.474956,0.428571,0.380000,1.0,0.0
2797,0.231637,0.002989,0.474648,0.281366,0.884615,0.911096,0.775340,0.982969,0.266071,1.114639,0.285714,0.390000,1.0,0.0
2798,0.019910,0.010449,0.248182,0.139128,0.192308,0.388282,0.613336,0.992426,0.361401,-0.105376,0.571429,0.400000,1.0,0.0


Q. What is the mean of the transformed test data (features only)?
Note : Compute the mean of the whole feature matrix i.e. mean of all values in the transformed test feature matrix

In [393]:
print(k1)
print(k2)

0.3861745655097562
0.38374582190073603
